## Load the digits dataset and evolutionary_search

In [1]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
import sklearn.datasets
import numpy as np
import pandas as pd

data = sklearn.datasets.load_digits()
X = data["data"]
y = data["target"]

# make it a 2-class problem by only classifying the digit "5" vs the rest
y = np.array([1 if yy == 5 else 0 for yy in y])

X.shape, y.shape

((1797, 64), (1797,))

In [2]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

## Train an SVM with RBF kernel

### Using conventional GridSearchCV

Parameter grid: 625 parameter combinations

In [3]:
paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=25, base=10),
             "gamma" : np.logspace(-9, 9, num=25, base=10)}
print("Size: ", len(paramgrid["kernel"])*len(paramgrid["C"])*len(paramgrid["gamma"]))

Size:  625


In [4]:
cv = GridSearchCV(estimator=SVC(),
                  param_grid=paramgrid,
                  scoring="accuracy",
                  cv=StratifiedKFold(n_splits=2),
                  verbose=1,n_jobs=-1)
%time cv.fit(X, y)

Fitting 2 folds for each of 625 candidates, totalling 1250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 854 tasks      | elapsed:   36.5s


Wall time: 1min 7s


[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed:  1.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jo...
       1.00000000e-03, 5.62341325e-03, 3.16227766e-02, 1.77827941e-01,
       1.00000000e+00, 5.62341325e+00, 3.16227766e+01, 1.77827941e+02,
       1.00000000e+03, 5.62341325e+03, 3.16227766e+04, 1.77827941e+05,
       1.00000000e+06, 5.62341325e+06, 3.16227766e+07, 1.77827941e+08,
       1.00000000e+09]),
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

### Best score + params

In [5]:
cv.best_score_, cv.best_params_

(0.9894268224819143, {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'})

An example of the "cannonical" cv_results_ table in sklearn:

In [6]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
308,0.021015,0.001501,0.020766,0.001250,1,0.001,rbf,"{'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'}",0.986652,0.992205,0.989427,0.002777,1
358,0.024517,0.006004,0.014761,0.000751,31.6228,0.001,rbf,"{'C': 31.622776601683793, 'gamma': 0.001, 'ker...",0.986652,0.991091,0.988870,0.002220,2
483,0.023767,0.005253,0.014760,0.001251,177828,0.001,rbf,"{'C': 177827.94100389228, 'gamma': 0.001, 'ker...",0.986652,0.991091,0.988870,0.002220,2
458,0.019263,0.000751,0.014010,0.000500,31622.8,0.001,rbf,"{'C': 31622.776601683792, 'gamma': 0.001, 'ker...",0.986652,0.991091,0.988870,0.002220,2
408,0.021265,0.002751,0.015510,0.000001,1000,0.001,rbf,"{'C': 1000.0, 'gamma': 0.001, 'kernel': 'rbf'}",0.986652,0.991091,0.988870,0.002220,2


### Using RandomizedSearchCV

Same parameter space, but only test 250 random combinations.

In [7]:
cv = RandomizedSearchCV(estimator=SVC(),
                        param_distributions=paramgrid,
                        n_iter=250,
                        scoring="accuracy",
                        cv=StratifiedKFold(n_splits=2),
                        verbose=1,n_jobs=-1)
%time cv.fit(X, y)

Fitting 2 folds for each of 250 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 350 tasks      | elapsed:   17.6s


Wall time: 25.3 s


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   25.1s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovr',
                                 degree=3, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, probability=False,
                                 random_state=None, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='warn...
       1.00000000e-03, 5.62341325e-03, 3.16227766e-02, 1.77827941e-01,
       1.00000000e+00, 5.62341325e+00, 3.16227766e+01, 1.77827941e+02,
       1.00000000e+03, 5.62341325e+03, 3.16227766e+04, 1.77827941e+05,
       1.00000000e+06, 5.62341325e+06, 3.16227766e+07, 1.77827941e+08,
       1.00000000e+09]),
                                        'kernel': ['rbf']},
                   pre_dispatch='2*n_

### Best score + params

In [8]:
cv.best_score_, cv.best_params_

(0.9888703394546466, {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000.0})

An example of the "cannonical" cv_results_ table in sklearn:

In [9]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_gamma,param_C,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
25,0.022265,7.500648e-04,0.016012,0.000501,rbf,0.001,1.77828e+08,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1778279...",0.986652,0.991091,0.98887,0.00222,1
33,0.022015,3.576279e-07,0.016763,0.000751,rbf,0.001,177.828,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 177.827...",0.986652,0.991091,0.98887,0.00222,1
36,0.027018,6.004453e-03,0.021516,0.004004,rbf,0.001,1e+06,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000000.0}",0.986652,0.991091,0.98887,0.00222,1
243,0.027018,1.000786e-02,0.013510,0.001001,rbf,0.001,31622.8,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 31622.7...",0.986652,0.991091,0.98887,0.00222,1
44,0.019013,1.501560e-03,0.013510,0.001001,rbf,0.001,5.62341,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 5.62341...",0.986652,0.991091,0.98887,0.00222,1


### Using EvolutionaryAlgorithmSearchCV

Again same parameter space, optimize for 10 generations.

In [10]:
from multiprocessing.pool import Pool
if __name__=="__main__":
    pool2 = Pool(4)
    cv = EvolutionaryAlgorithmSearchCV(estimator=SVC(),
                                       params=paramgrid,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=3,
                                       generations_number=10, n_jobs=pool2.map)
    %time cv.fit,(X, y)


Wall time: 0 ns


### Best score + params

In [11]:
cv.best_score_, cv.best_params_

(None, None)

Our cv_results_ table (note, includes all individuals with their mean, max, min, and std test score).

In [12]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

KeyError: 'mean_test_score'